# Baseline Model Train

In [ ]:
main_path = '/content/images/'

train_paths, test_paths = [], []

for filename in train_df.image:
  train_paths.append(main_path + filename + '.jpg')

for filename in test_df.image:
  test_paths.append(main_path + filename + '.jpg')

train_paths = np.array(train_paths)
test_paths = np.array(test_paths)

train_labels = train_df.label.values
test_labels = test_df.label.values

len(train_paths), len(train_labels), len(test_paths), len(test_labels)
######################################################################################################################################
s = np.arange(len(train_paths))
np.random.shuffle(s)

train_paths = train_paths[s]
train_labels = train_labels[s]
X_train, X_val, y_train, y_val = train_test_split(train_paths, train_labels, test_size=0.2, random_state=42)
#######################################################################################################################################

ss = s = np.arange(len(test_paths))
np.random.shuffle(ss)

test_paths = test_paths[ss]
test_labels = test_labels[ss]

def preprocessing(path, label):
  img = tf.io.read_file(path)
  img = tf.io.decode_jpeg(img)
  img = tf.image.resize(img, (256,256))
  img = img/255
  return img, label

batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

train_dataset = train_dataset.shuffle(len(train_paths))
train_dataset = train_dataset.map(preprocessing).batch(batch_size).prefetch(1)
val_dataset = val_dataset.map(preprocessing).batch(batch_size).prefetch(1)
test_dataset = test_dataset.map(preprocessing).batch(batch_size).prefetch(1)

In [ ]:
model = Sequential()
model.add(layers.Conv2D(128,3,padding='same', input_shape=[256,256,3]))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(128,3,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(256,3,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(512,3,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

opt = keras.optimizers.SGD()
# opt = keras.optimizers.SGD()
loss = keras.losses.SparseCategoricalCrossentropy()


es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
model.fit(train_dataset, validation_data=val_dataset, epochs=40, verbose=1, callbacks=[es])

# Model Evaluation

In [ ]:
def pred_preprocessing(path, label):
  img = tf.io.read_file(path)
  img = tf.io.decode_jpeg(img)
  img = tf.image.resize(img, (256,256))
  img = img/255
  return img

batch_size = 16


pred_test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
pred_test_dataset = pred_test_dataset.map(pred_preprocessing).batch(batch_size).prefetch(1)

y_pred = model.predict(pred_test_dataset)
label_pred = np.argmax(y_pred, axis=1)

In [ ]:
precision = precision_score(test_labels, label_pred, average='macro')
recall = recall_score(test_labels, label_pred, average= "macro")
F1 = 2 * (precision * recall) / (precision + recall)


print(f'정밀도 : {precision:.4f}')
print(f'재현율 : {recall:.4f}')
print(f'f1_score : {F1:.4f}')